* Looks for linear, quad and cubic trends in cat var
* n.b: wise to use only when ordinal cat var has equally spaced levels

In [1]:
#8th contrast 

import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [2]:
car_data = pd.read_csv("./datasets/auto-mpg.csv", na_values="?")

car_data.sample(8)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
109,21.0,4,140.0,72.0,2401,19.5,73,1,chevrolet vega
261,18.1,6,258.0,120.0,3410,15.1,78,1,amc concord d/l
293,31.9,4,89.0,71.0,1925,14.0,79,2,vw rabbit custom
268,27.2,4,119.0,97.0,2300,14.7,78,3,datsun 510
313,28.0,4,151.0,90.0,2678,16.5,80,1,chevrolet citation
54,35.0,4,72.0,69.0,1613,18.0,71,3,datsun 1200
35,17.0,6,250.0,100.0,3329,15.5,71,1,chevrolet chevelle malibu
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup


In [3]:
# using just mpg and horsepower for work
car_data = car_data[["mpg", "horsepower"]]

car_data.dropna(inplace = True)
car_data.reset_index(inplace = True, drop = True)

car_data.sample(6)

<ipython-input-3-4c08e6aa21d6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_data.dropna(inplace = True)


,mpg,horsepower
280,22.3,88.0
86,13.0,145.0
28,9.0,193.0
142,31.0,52.0
166,23.0,83.0
181,25.0,81.0


In [5]:
# horse power is inversely corr with mpg
car_data.corr()

,mpg,horsepower
mpg,1.000000,-0.778427
horsepower,-0.778427,1.000000


In [6]:
# cretes 4 buckets that are equally spaced, n.b 3 bin edges = 4 buckets
_, bin_edges = np.histogram(car_data["horsepower"], 3)

In [7]:
# we bin edge(firt bucket is hp values of 46 - 107.33333)
bin_edges

array([ 46.        , 107.33333333, 168.66666667, 230.        ])

In [8]:
# coverts equally spaced bins to cat values
hp_cat = np.digitize(car_data.horsepower, bin_edges, True)

hp_cat[:30]

array([2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 3, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 2, 1, 3, 3, 3, 3, 1], dtype=int64)

In [9]:
# creat col in df for this cat values
car_data["hp_cat"] = hp_cat

car_data.sample(6)

,mpg,horsepower,hp_cat
324,43.4,48.0,1
166,23.0,83.0,1
259,18.1,120.0,2
269,23.2,105.0,1
167,20.0,100.0,1
377,38.0,67.0,1


In [10]:
# no need for horsePower col again
car_data.drop(columns = ["horsepower"], inplace = True)

In [12]:
# group data by cat levels
car_data_grouped = car_data.groupby("hp_cat")
car_data_grouped.head()

,mpg,hp_cat
0,18.0,2
1,15.0,2
2,18.0,2
3,16.0,2
4,17.0,2
5,15.0,3
6,14.0,3
7,14.0,3
8,14.0,3
9,15.0,3


In [13]:
# mean mpg for all bins cat
car_data_grouped.mean()

,mpg
hp_cat,
0,26.000000
1,27.186275
2,17.289320
3,13.296875


In [14]:
# mean of cat means
car_data_grouped.mean().mean()

mpg    20.943117
dtype: float64

**statsmodels for orthogonal polynomoal encoding**

In [15]:
mod = ols("mpg ~ C(hp_cat, Poly)", data = car_data)

result = mod.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.453
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     106.9
Date:                Tue, 12 Jan 2021   Prob (F-statistic):           1.80e-50
Time:                        15:50:49   Log-Likelihood:                -1243.1
No. Observations:                 392   AIC:                             2494.
Df Residuals:                     388   BIC:                             2510.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    20.9431      1.070     19.577      0.000      18.840      23.046
C(hp_cat, Poly).Linear      -10.7345      2.838     -3.782      0.000     -16.315      -5.154
C(hp_cat, Poly).Quadratic    -2.5894      2.140     -1.210      0.227      -6.796       1.617
C(hp_cat, Poly).Cubic         3.7986      1.048      3.624      0.000       1.738       5.859
==============================================================================
Omnibus:                       15.471   Durbin-Watson:                   0.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.178
Skew:                           0.480   Prob(JB):                     0.000307
Kurtosis:                       3.266   Cond. No.                         14.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**OrthpolyEncoding using stats models**

In [16]:
from patsy.contrasts import Poly

In [17]:
levels = [0, 1, 2, 3]

In [18]:
contrast_with_int = Poly().code_with_intercept(levels)

contrast_with_int

ContrastMatrix(array([[ 1.        , -0.67082039,  0.5       , -0.2236068 ],
                      [ 1.        , -0.2236068 , -0.5       ,  0.67082039],
                      [ 1.        ,  0.2236068 , -0.5       , -0.67082039],
                      [ 1.        ,  0.67082039,  0.5       ,  0.2236068 ]]),
               ['.Constant', '.Linear', '.Quadratic', '.Cubic'])

In [19]:
contrast_without_int = Poly().code_without_intercept(levels)

contrast_without_int

ContrastMatrix(array([[-0.67082039,  0.5       , -0.2236068 ],
                      [-0.2236068 , -0.5       ,  0.67082039],
                      [ 0.2236068 , -0.5       , -0.67082039],
                      [ 0.67082039,  0.5       ,  0.2236068 ]]),
               ['.Linear', '.Quadratic', '.Cubic'])

In [20]:
car_data_contrast = contrast_without_int.matrix[car_data.hp_cat - 0, :]

car_data_contrast[:6]

array([[ 0.2236068 , -0.5       , -0.67082039],
       [ 0.2236068 , -0.5       , -0.67082039],
       [ 0.2236068 , -0.5       , -0.67082039],
       [ 0.2236068 , -0.5       , -0.67082039],
       [ 0.2236068 , -0.5       , -0.67082039],
       [ 0.67082039,  0.5       ,  0.2236068 ]])

In [21]:
# df with encoded values
car_data_contrast_df = pd.DataFrame(car_data_contrast,
                                    columns = ["linear", "quadratic", "cubic"])

car_data_contrast_df.sample(7)

,linear,quadratic,cubic
50,-0.223607,-0.5,0.67082
214,-0.223607,-0.5,0.67082
295,-0.223607,-0.5,0.67082
3,0.223607,-0.5,-0.67082
233,-0.223607,-0.5,0.67082
234,-0.223607,-0.5,0.67082
190,-0.223607,-0.5,0.67082


In [22]:
# concat encoded df with main df
car_data_enc = pd.concat([car_data, car_data_contrast_df], axis = "columns")

car_data_enc.sample(6)

,mpg,hp_cat,linear,quadratic,cubic
194,24.5,1,-0.223607,-0.5,0.67082
124,20.0,1,-0.223607,-0.5,0.67082
24,21.0,1,-0.223607,-0.5,0.67082
48,23.0,1,-0.223607,-0.5,0.67082
232,24.5,1,-0.223607,-0.5,0.67082
138,14.0,2,0.223607,-0.5,-0.67082


In [24]:
X = car_data_enc.drop(columns = ["mpg", "hp_cat"], axis = 1)
y = car_data_enc["mpg"]

In [26]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(fit_intercept = True)
linear_model.fit(X, y)

print("Training_score: ", linear_model.score(X, y))

Training_score:  0.45261121476571653


In [27]:
linear_model.coef_

array([-10.73454153,  -2.58935995,   3.79857355])

In [28]:
linear_model.intercept_

20.943117474538344